In [31]:
import pandas as pd
import glob, os 
import pickle
import numpy as np
import stdb
from obspy.geodetics.base import gps2dist_azimuth

In [2]:
path = '/Volumes/home/Research/STEP/02_Station_result_csv'
pklpath = '/Volumes/home/Research/STEP/01_Split'

previous = ['DGRL','CMCY','MAKU']

In [21]:
def Calc_rho(RCdt, SCdt):
    rho = RCdt/SCdt 
    return rho
def Calc_Phi(RCPhi, SCPhi):
    Phi = max (abs(RCPhi-SCPhi), abs(SCPhi-RCPhi))
    if Phi > 90: Phi = 180 - Phi
    return Phi

In [33]:
totaldf = pd.DataFrame()
for stapkl in sorted(glob.glob(f'{pklpath}/*pkl')):
    station = pickle.load(open(stapkl, "rb"))
    values = [value for keys, value in station.items() ][0]
    NET = values['network']; STA = values['station']
    stlat = values['latitude']; stlon=values['longitude']
    if STA not in previous:
        dff_ = pd.DataFrame()
        bazlist = []
        for csvpath in glob.glob(f'{path}/*classify_220913/*{STA}*result_v2.csv'):
            print(csvpath)
            df = pd.read_csv(csvpath)
            df_ = df[df['Pick']==True]
            dff_ = pd.concat([dff_,df_])
            
        for i in range(len(dff_)):
            event = dff_['Event'].values[i]
            evlon = dff_['Ev_lon'].values[i]
            evlat = dff_['Ev_lat'].values[i]
            dist, az, baz = gps2dist_azimuth(evlat, evlon, stlat,stlon)
            bazlist.append(round(baz,2))
            Null = dff_['Null'].values[i]
            CpH = dff_['CpH'].values[i]

            
            if CpH >=0.8:
                dff_['Null'].values[i] = True
                dff_['SCPhi'].values[i] = np.NaN
                dff_['RCPhi'].values[i] = np.NaN
                dff_['SCdt'].values[i]  = np.NaN
                dff_['RCdt'].values[i]  = np.NaN 
                dff_['SCPhi_std'].values[i] = np.NaN
                dff_['RCPhi_std'].values[i] = np.NaN
                dff_['SCdt_std'].values[i]  = np.NaN
                dff_['RCdt_std'].values[i]  = np.NaN     
                dff_['Quality'].values[i]  = np.NaN   
            else:
                dff_['Null'].values[i] = False
                
                if Null == True and CpH<0.8:
                    SCphi = dff_['SCPhi'].values[i]
                    RCphi = dff_['RCPhi'].values[i]
                    SCdt = dff_['SCdt'].values[i]
                    RCdt = dff_['RCdt'].values[i]
                    rho =  Calc_rho(RCdt, SCdt)
                    PHI = Calc_Phi(RCphi, SCphi)

                    if rho > 1.2 or PHI>25:
                        if dff_['Quality'].values[i] != 'Poor':
                            print(event)
                            dff_['Quality'].values[i] = 'Poor'
                        
                        
        length=len(dff_)
        dff_ = dff_.drop(['SNRQ','SNRT','Pick','coeff'], axis=1)
        colname=dff_.columns.to_list()
        colname.insert(colname.index('Phase'),'Baz')
        colname.insert(colname.index('Event'),'Station')
        
        newdf = dff_.reindex(columns=colname)
        newdf['Station']=[f'{NET}.{STA}']*length
        newdf['Baz']=bazlist
        
        totaldf = pd.concat([totaldf,newdf],ignore_index=True)
        
totaldf.to_csv('catalog_nullNAN_v1121.csv')

/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify_220913/AB.GANJ_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify_220913/AB.GANJ_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify_220913/AB.QZX_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify_220913/AB.QZX_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify_220913/AB.ZKT_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify_220913/AB.ZKT_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classify_220913/AM.ALAV_split_result_v2.csv
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKS_classify_220913/AM.ALAV_split_result_v2.csv
20140411_202913
20140412_201438
20151217_194953
20181101_221951
/Volumes/home/Research/STEP/02_Station_result_csv/2010-2020_SKKS_classif

In [32]:
totaldf

,Station,Event,Ev_lat,Ev_lon,Depth,Mag,Baz,Phase,SCPhi,SCPhi_std,SCdt,SCdt_std,RCPhi,RCPhi_std,RCdt,RCdt_std,Null,Quality,CpH
0,AB.GANJ,20110824_174611,-7.6203,-74.5380,149.3,7.0,285.36,SKS,-88.64,22.00,0.7,0.95,61.36,22.25,0.3,0.98,True,Fair,0.87
1,AB.GANJ,20110830_065741,-6.3592,126.7502,467.2,6.9,100.91,SKS,-24.09,21.50,0.8,0.95,-35.09,44.75,0.7,0.98,False,Fair,0.73
2,AB.GANJ,20110903_044858,-56.5611,-26.9231,96.3,6.4,215.73,SKS,-59.27,44.75,2.9,0.98,71.73,44.75,0.3,0.98,True,Fair,0.85
3,AB.GANJ,20111014_033515,-6.6259,147.9311,43.2,6.5,87.33,SKS,-44.67,13.25,0.8,0.18,-47.67,18.00,0.8,0.35,False,Good,0.70
4,AB.GANJ,20111122_184816,-15.3083,-65.1634,560.3,6.6,272.10,SKS,88.10,44.75,2.2,0.98,46.10,44.75,0.2,0.98,True,Fair,0.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,TG.URAV,20120528_050723,-28.0210,-63.1134,591.1,6.7,258.41,SKS,2.41,7.50,2.2,0.75,42.41,44.75,0.7,0.98,True,Fair,0.88
1187,TG.URAV,20101215_112930,-7.2664,128.7939,140.4,6.0,98.22,SKS,22.22,3.25,3.9,0.35,79.22,44.75,0.3,0.98,True,Fair,0.88
1188,TG.URAV,20110824_174611,-7.6203,-74.5380,149.3,7.0,284.02,SKS,-85.98,7.75,1.8,0.65,55.02,44.75,0.5,0.98,True,Fair,0.82
1189,TG.URAV,20111028_185434,-14.5572,-76.1207,29.0,7.0,279.44,SKS,24.44,44.75,0.7,0.98,57.44,44.75,0.3,0.98,True,Fair,0.85
